In [ ]:
!pip install streamlit faker
!pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 117.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile app.py
import streamlit as st
from PIL import Image
import numpy as np
import tensorflow as tf
import struct
from faker import Faker
from keras.models import load_model

# Load models
generator = load_model('/content/drive/MyDrive/CIP/Saved_Models/new2/generator_model_updated_new.keras')
discriminator = load_model('/content/drive/MyDrive/CIP/Saved_Models/new/discriminator_model_updated_new.keras')
extractor = load_model('/content/drive/MyDrive/CIP/Saved_Models/new2/extractor_model_updated_new.keras')

# Global constants
MAX_BITS = 16384
fake = Faker()

# Session state setup
if "cover_image" not in st.session_state:
    st.session_state.cover_image = None
if "stego_image" not in st.session_state:
    st.session_state.stego_image = None
if "binary_message" not in st.session_state:
    st.session_state.binary_message = None

# Functions
def preprocess_image(img):
    img = img.convert("L").resize((256, 256))  # Grayscale and resize
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=-1)
    return img_array

def generate_stego_image(cover_image, message):
    cover_image = np.expand_dims(cover_image, axis=0)
    message = message.reshape(1, -1)
    stego_image = generator.predict([cover_image, message])
    return np.squeeze(stego_image, axis=0)

def test_stego_image(stego_image):
    stego_image = np.expand_dims(stego_image / 255.0, axis=0)
    prediction = discriminator.predict(stego_image)
    label = "Stego" if prediction[0][0] > 0.5 else "Real"
    confidence = prediction[0][0]
    return label, confidence

def extract_message(stego_image):
    stego_image = np.expand_dims(stego_image / 255.0, axis=0)
    extracted = extractor.predict(stego_image)
    extracted = np.round(extracted).astype(np.uint8).squeeze()
    return extracted

def generate_patient_info():
    patient_id = int(fake.random_number(digits=8))
    age = fake.random_int(min=18, max=99)
    gender = fake.random_element(elements=("M", "F", "O")).encode()
    blood_type = fake.random_element(elements=("A+", "A-", "B+", "B-", "O+", "O-", "AB+", "AB-")).encode()
    allergies = ",".join(fake.words(3))[:30].encode()
    chronic_conditions = ",".join(fake.words(5))[:50].encode()
    medications = ",".join(fake.sentences(3))[:80].encode()
    treatments = ",".join(fake.sentences(3))[:80].encode()
    visits = ",".join([fake.date_this_decade().isoformat() for _ in range(3)])[:50].encode()
    hemoglobin = round(fake.pyfloat(min_value=10.0, max_value=18.0, right_digits=1), 1)
    cholesterol = round(fake.pyfloat(min_value=100.0, max_value=250.0, right_digits=1), 1)
    systolic_bp = fake.random_int(90, 140)
    diastolic_bp = fake.random_int(60, 90)
    heart_rate = fake.random_int(60, 100)

    binary_data = struct.pack(
        ">I B c 2s 30s 50s 80s 80s 50s f f H H H",
        patient_id, age, gender, blood_type,
        allergies, chronic_conditions, medications, treatments, visits,
        hemoglobin, cholesterol, systolic_bp, diastolic_bp, heart_rate
    )
    # st.write(binary_data)
    bit_array = np.unpackbits(np.frombuffer(binary_data, dtype=np.uint8))
    return np.pad(bit_array, (0, max(0, MAX_BITS - len(bit_array))), 'constant')

def decode_patient_info(binary_message):
    byte_array = np.packbits(binary_message)
    fmt = ">I B c 2s 30s 50s 80s 80s 50s f f H H H"
    buffer_size = struct.calcsize(fmt)
    unpacked = struct.unpack(fmt, byte_array[:buffer_size])
    return {
        "patient_id": unpacked[0],
        "age": unpacked[1],
        "gender": unpacked[2].decode(),
        "blood_type": unpacked[3].decode(),
        "allergies": unpacked[4].decode().strip(),
        "chronic_conditions": unpacked[5].decode().strip(),
        "current_medications": unpacked[6].decode().strip(),
        "recent_treatments": unpacked[7].decode().strip(),
        "hospital_visits": unpacked[8].decode().strip(),
        "lab_results": {
            "hemoglobin": unpacked[9],
            "cholesterol": unpacked[10],
            "blood_pressure": f"{unpacked[11]}/{unpacked[12]}",
            "heart_rate": unpacked[13]
        }
    }

# Streamlit UI
st.title("GAN-based Image Steganography Tool")

# Sidebar - Operation Tabs
tab = st.sidebar.radio("Select Operation", ["Embed Message", "Discriminate Image", "Extract Message"])

# Embed Message
if tab == "Embed Message":
    st.title("Cover Image")
    uploaded_image = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
    if uploaded_image:
        st.session_state.cover_image = Image.open(uploaded_image)
        st.image(st.session_state.cover_image, caption="Uploaded Image", use_container_width=True)

    if st.button("Generate Patient Info"):
        st.session_state.binary_message = generate_patient_info()
        st.success("Patient Info Generated!")

    if st.session_state.binary_message is not None:
        st.text_area("Binary Message", value=st.session_state.binary_message.tolist(), height=150)

    if st.button("Embed"):
        if st.session_state.cover_image is None:
            st.warning("Upload a cover image first.")
        elif st.session_state.binary_message is None:
            st.warning("Generate patient info first.")
        else:
            image_array = preprocess_image(st.session_state.cover_image)
            stego = generate_stego_image(image_array, st.session_state.binary_message)
            st.session_state.stego_image = stego
            st.image(stego, caption="Stego Image", use_container_width=True)
            st.download_button("Download Stego Image", data=stego.tobytes(),
                               file_name="stego_image.png", mime="image/png")
            st.success("Stego image generated and stored!")

# Discriminate Image
elif tab == "Discriminate Image":
    if st.session_state.stego_image is not None:
        st.image(st.session_state.stego_image, caption="Stego Image", use_container_width=True)
        if st.button("Run Discriminator"):
            label, confidence = test_stego_image(st.session_state.stego_image)
            st.info(f"Prediction: **{label}** with confidence {confidence:.4f}")
    else:
        st.warning("No stego image available. Please embed a message first.")

# Extract Message
elif tab == "Extract Message":
    if st.session_state.stego_image is not None:
        st.image(st.session_state.stego_image, caption="Stego Image", use_container_width=True)
        if st.button("Extract Message"):

            message = extract_message(st.session_state.stego_image)
            decoded = decode_patient_info(message)
            st.json(decoded)
            st.success("Message extracted!")
    else:
        st.warning("No stego image available. Please embed a message first.")


Overwriting app.py


In [ ]:
!python -m streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.240.246.188:8501

  Stopping...
  Stopping...
